# LangChainの準備

In [ ]:
# パッケージのインストール
!pip install langchain==0.0.181
!pip install openai

In [ ]:
# 環境変数の準備
import os
os.environ["OPENAI_API_KEY"] = "<OpenAI_APIのAPIキー>"

In [ ]:
# パッケージのインストール
!pip install tiktoken
!pip install faiss-gpu

# プロンプトテンプレートの作成

In [ ]:
from langchain.prompts import PromptTemplate

# 入力変数のないプロンプトテンプレートの作成
no_input_prompt = PromptTemplate(
    input_variables=[],
    template="かっこいい動物といえば？"
)

# プロンプトの作成
print(no_input_prompt.format())

In [ ]:
from langchain.prompts import PromptTemplate

# 1つの入力変数を持つプロンプトテンプレートの作成
one_input_prompt = PromptTemplate(
    input_variables=["content"],
    template="かっこいい{content}といえば？"
)

# プロンプトの作成
print(one_input_prompt.format(content="動物"))

In [ ]:
from langchain.prompts import PromptTemplate

# 複数の入力変数を持つプロンプトテンプレートの作成
multiple_input_prompt = PromptTemplate(
    input_variables=["adjective", "content"],
    template="{adjective}{content}といえば？"
)

# プロンプトの作成
print(multiple_input_prompt.format(adjective="かっこいい", content="動物"))

In [ ]:
from langchain.prompts import PromptTemplate

# jinja2を使ったプロンプトテンプレートの作成
jinja2_prompt = PromptTemplate(
    input_variables=["items"],
    template_format="jinja2",
    template="""
{% for item in items %}
Q: {{ item.question }}
A: {{ item.answer }}
{% endfor %}
"""
)

# プロンプトの作成
items=[
    {"question": "foo", "answer": "bar"},
    {"question": "1", "answer": "2"}
]
print(jinja2_prompt.format(items=items))

# 回答例を含むプロンプトテンプレートの作成

In [ ]:
from langchain.prompts import FewShotPromptTemplate

# 回答例の準備
examples = [
    {"input": "明るい", "output": "暗い"},
    {"input": "おもしろい", "output": "つまらない"},
]

# プロンプトテンプレートの作成
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template="入力: {input}\n出力: {output}",
)

# 回答例を含むプロンプトテンプレートの作成
prompt_from_string_examples = FewShotPromptTemplate(
    examples=examples, # 回答例
    example_prompt=example_prompt,  # プロンプトテンプレート
    prefix="すべての入力の反意語を与えてください",  # プレフィックス
    suffix="入力: {adjective}\n出力:",  # サフィックス
    input_variables=["adjective"],  # 入力変数
    example_separator="\n\n"  # セパレータ

)

# プロンプトの作成
print(prompt_from_string_examples.format(adjective="大きい"))

# LengthBasedExampleSelector

In [ ]:
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

# 回答例の準備
examples = [
    {"input": "明るい", "output": "暗い"},
    {"input": "おもしろい", "output": "つまらない"},
    {"input": "エネルギッシュ", "output": "無気力"},
    {"input": "高い", "output": "低い"},
    {"input": "明るい", "output": "暗い"},
    {"input": "早い", "output": "遅い"},
]

# プロンプトテンプレートの作成
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template="入力: {input}\n出力: {output}",
)

# LengthBasedExampleSelectorの作成
example_selector = LengthBasedExampleSelector(
    examples=examples,  # 回答例
    example_prompt=example_prompt,  # プロンプトテンプレート
    max_length=10,  # 最大長
)

# FewShotPromptTemplateの準備
prompt_from_string_examples = FewShotPromptTemplate(
    example_selector=example_selector,  # ExampleSelector
    example_prompt=example_prompt,
    prefix="すべての入力の反意語を与えてください",
    suffix="入力: {adjective}\n出力:",
    input_variables=["adjective"],
    example_separator="\n\n"

)

# プロンプトの作成
print(prompt_from_string_examples.format(adjective="大きい"))

# SemanticSimilarityExampleSelector

In [ ]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate

# 回答例の準備
examples = [
    {"input": "明るい", "output": "暗い"},
    {"input": "おもしろい", "output": "つまらない"},
    {"input": "エネルギッシュ", "output": "無気力"},
    {"input": "高い", "output": "低い"},
    {"input": "明るい", "output": "暗い"},
    {"input": "早い", "output": "遅い"},
]

# プロンプトテンプレートの作成
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template="入力: {input}\n出力: {output}",
)

# SemanticSimilarityExampleSelectorの作成
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,  # 回答例
    embeddings=OpenAIEmbeddings(),  # 埋め込み生成のクラス
    vectorstore_cls=FAISS,  # 埋め込み類似検索のクラス
    k=3  # 回答例の数
)

# FewShotPromptTemplateの作成
prompt_from_string_examples = FewShotPromptTemplate(
    example_selector=example_selector,  # ExampleSelector
    example_prompt=example_prompt,
    prefix="すべての入力の反意語を与えてください",
    suffix="入力: {adjective}\n出力:",
    input_variables=["adjective"],
    example_separator="\n\n"

)

# プロンプトの作成
print(prompt_from_string_examples.format(adjective="大きい"))

# MaxMarginalRelevanceExampleSelector

In [ ]:
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate

# 回答例の準備
examples = [
    {"input": "明るい", "output": "暗い"},
    {"input": "おもしろい", "output": "つまらない"},
    {"input": "エネルギッシュ", "output": "無気力"},
    {"input": "高い", "output": "低い"},
    {"input": "明るい", "output": "暗い"},
    {"input": "早い", "output": "遅い"},
]

# プロンプトテンプレートの作成
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template="入力: {input}\n出力: {output}",
)

# MaxMarginalRelevanceExampleSelectorの作成
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples=examples,  # 回答例
    embeddings=OpenAIEmbeddings(),  # 埋め込み生成のクラス
    vectorstore_cls=FAISS,  # 埋め込み類似検索のクラス
    k=3  # 回答例の数
)

# FewShotPromptTemplateの準備
prompt_from_string_examples = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="すべての入力の反意語を与えてください",
    suffix="入力: {adjective}\n出力:",
    input_variables=["adjective"],
    example_separator="\n\n"

)
print(prompt_from_string_examples.format(adjective="大きい"))